In [2]:
#!pip install python-pptx
#!pip install edge-tts

In [3]:
import collections
import collections.abc
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN, MSO_ANCHOR
from pptx.dml.color import RGBColor
from pptx.oxml.xmlchemy import OxmlElement
from pptx.oxml.ns import qn

# ชื่อฟอนต์ (ต้องมีในเครื่องคอมพิวเตอร์ที่เปิดไฟล์)
FONT_NAME = "TH Sarabun New"

In [4]:
#Set5
questions_data = [
    {
        "id": 1,
        "narrator": "What does the woman imply?",
        "script_eng": "(Man): I'm thinking of buying that used car we saw yesterday.\n(Woman): Are you sure? It seemed to have a lot of mileage on it.",
        "narrator_thai": "ผู้หญิงบอกเป็นนัยว่าอะไร?",
        "script_thai": "(Man): ผมกำลังคิดว่าจะซื้อรถมือสองคันที่เราไปดูเมื่อวาน\n(Woman): คุณแน่ใจเหรอ? ดูเหมือนรถคันนั้นจะเลขไมล์เยอะมากเลยนะ (วิ่งมาเยอะ)",
        "choices_eng": [
            "(A) The man should buy the car.",
            "(B) The car is in good condition.",
            "(C) The car might not be a good purchase.",
            "(D) She wants to buy the car herself."
        ],
        "choices_thai": [
            "(ผู้ชายควรซื้อรถคันนั้น)",
            "(รถอยู่ในสภาพดี)",
            "(รถคันนั้นอาจจะไม่ใช่การซื้อที่คุ้มค่านัก)",
            "(เธออยากซื้อรถคันนั้นเอง)"
        ],
        "answer_text": "เฉลย: (C)\nอธิบาย: ผู้หญิงทักท้วงว่า 'Are you sure?' และชี้ข้อเสียเรื่อง 'lot of mileage' (เลขไมล์เยอะ) เป็นนัยว่ารถสภาพไม่ดีและไม่น่าซื้อ"
    },
    {
        "id": 2,
        "narrator": "What does the man mean?",
        "script_eng": "(Woman): I'm having trouble with this calculus problem. It's so confusing.\n(Man): Don't look at me. Math was never my strong suit.",
        "narrator_thai": "ผู้ชายหมายความว่าอย่างไร?",
        "script_thai": "(Woman): ฉันกำลังงงกับโจทย์แคลคูลัสข้อนี้ มันสับสนจัง\n(Man): อย่ามองผมสิ (อย่าหวังพึ่งผม) วิชาเลขไม่เคยเป็นจุดแข็งของผมเลย",
        "choices_eng": [
            "(A) He is very good at math.",
            "(B) He can help her with the problem.",
            "(C) He doesn't like the woman.",
            "(D) He is not good at math."
        ],
        "choices_thai": [
            "(เขาเก่งเลขมาก)",
            "(เขาสามารถช่วยเธอทำโจทย์ได้)",
            "(เขาไม่ชอบผู้หญิงคนนี้)",
            "(เขาไม่เก่งเลข)"
        ],
        "answer_text": "เฉลย: (D)\nอธิบาย: สำนวน 'never my strong suit' หมายความว่า ไม่เก่งหรือไม่ถนัดเรื่องนั้นๆ"
    },
    {
        "id": 3,
        "narrator": "What will the woman probably do?",
        "script_eng": "(Man): The elevator is broken again. We'll have to take the stairs to the 10th floor.\n(Woman): You go ahead. I'll wait for the next elevator in the lobby.",
        "narrator_thai": "ผู้หญิงน่าจะทำอะไร?",
        "script_thai": "(Man): ลิฟต์เสียอีกแล้ว เราต้องเดินขึ้นบันไดไปชั้น 10 กันแล้วล่ะ\n(Woman): คุณไปก่อนเลย ฉันจะรอลิฟต์ตัวถัดไปที่ล็อบบี้",
        "choices_eng": [
            "(A) Walk up the stairs.",
            "(B) Fix the elevator.",
            "(C) Wait for another elevator.",
            "(D) Leave the building."
        ],
        "choices_thai": [
            "(เดินขึ้นบันได)",
            "(ซ่อมลิฟต์)",
            "(รอลิฟต์ตัวอื่น)",
            "(ออกจากตึก)"
        ],
        "answer_text": "เฉลย: (C)\nอธิบาย: ผู้หญิงพูดชัดเจนว่า 'I'll wait for the next elevator' (ฉันจะรอลิฟต์ตัวถัดไป)"
    },
    {
        "id": 4,
        "narrator": "What does the man suggest?",
        "script_eng": "(Woman): I left my wallet in the classroom, but the door is locked now.\n(Man): Security should have a key. Let's go find an officer.",
        "narrator_thai": "ผู้ชายแนะนำว่าอะไร?",
        "script_thai": "(Woman): ฉันลืมกระเป๋าตังค์ไว้ในห้องเรียน แต่ตอนนี้ประตูล็อคไปแล้ว\n(Man): ฝ่ายรักษาความปลอดภัยน่าจะมีกุญแจนะ เราไปหาเจ้าหน้าที่กันเถอะ",
        "choices_eng": [
            "(A) Break the door open.",
            "(B) Ask a security officer for help.",
            "(C) Leave the wallet there.",
            "(D) Call the professor."
        ],
        "choices_thai": [
            "(พังประตูเข้าไป)",
            "(ขอความช่วยเหลือจากเจ้าหน้าที่ รปภ.)",
            "(ทิ้งกระเป๋าตังค์ไว้ที่นั่น)",
            "(โทรหาอาจารย์)"
        ],
        "answer_text": "เฉลย: (B)\nอธิบาย: ผู้ชายแนะนำว่า 'Let's go find an officer' (ไปหาเจ้าหน้าที่กันเถอะ) เพื่อให้มาช่วยเปิดประตู"
    },
    {
        "id": 5,
        "narrator": "What does the woman imply about the presentation?",
        "script_eng": "(Man): How did your presentation go this morning?\n(Woman): Well, let's just say I'm glad it's over.",
        "narrator_thai": "ผู้หญิงบอกเป็นนัยเกี่ยวกับการนำเสนองานว่าอย่างไร?",
        "script_thai": "(Man): การนำเสนองานเมื่อเช้าเป็นยังไงบ้าง?\n(Woman): เอาเป็นว่า ฉันดีใจที่มันจบลงสักที",
        "choices_eng": [
            "(A) It went extremely well.",
            "(B) She wants to do it again.",
            "(C) It didn't go very well.",
            "(D) It was too short."
        ],
        "choices_thai": [
            "(มันผ่านไปได้ด้วยดีมากๆ)",
            "(เธออยากทำมันอีกครั้ง)",
            "(มันไม่ค่อยดีเท่าไหร่)",
            "(มันสั้นเกินไป)"
        ],
        "answer_text": "เฉลย: (C)\nอธิบาย: วลี 'I'm glad it's over' (ดีใจที่มันจบไปซะที) มักใช้เมื่อเหตุการณ์นั้นยากลำบากหรือไม่น่าอภิรมย์ สื่อเป็นนัยว่าผลลัพธ์ไม่ค่อยดีนัก"
    },
    {
        "id": 6,
        "narrator": "What does the man mean?",
        "script_eng": "(Woman): Do you want to try the new seafood place tonight?\n(Man): I'm not really a big fan of fish.",
        "narrator_thai": "ผู้ชายหมายความว่าอย่างไร?",
        "script_thai": "(Woman): คืนนี้คุณอยากไปลองร้านอาหารทะเลเปิดใหม่ไหม?\n(Man): ผมไม่ค่อยชอบกินปลาเท่าไหร่นะ",
        "choices_eng": [
            "(A) He loves seafood.",
            "(B) He wants to go to the restaurant.",
            "(C) He is a fan of the restaurant.",
            "(D) He prefers not to eat seafood."
        ],
        "choices_thai": [
            "(เขาชอบอาหารทะเลมาก)",
            "(เขาอยากไปที่ร้านอาหารนั้น)",
            "(เขาเป็นแฟนคลับของร้านนั้น)",
            "(เขาเลือกที่จะไม่กินอาหารทะเล)"
        ],
        "answer_text": "เฉลย: (D)\nอธิบาย: สำนวน 'not a big fan of...' แปลว่า ไม่ค่อยชอบ ดังนั้นเขาจึงไม่อยากไปกิน (Prefers not to eat)"
    },
    {
        "id": 7,
        "narrator": "What does the woman suggest the man do?",
        "script_eng": "(Man): I've got a terrible headache, and I have a chemistry lab in an hour.\n(Woman): If I were you, I’d take some aspirin and lie down for a bit.",
        "narrator_thai": "ผู้หญิงแนะนำให้ผู้ชายทำอะไร?",
        "script_thai": "(Man): ผมปวดหัวจี๊ดเลย แล้วมีเรียนแล็บเคมีในอีกชั่วโมงนึงด้วย\n(Woman): ถ้าฉันเป็นคุณ ฉันจะกินยาแอสไพรินแล้วนอนพักสักหน่อย",
        "choices_eng": [
            "(A) Go to the chemistry lab immediately.",
            "(B) Take medicine and rest.",
            "(C) Study for the lab.",
            "(D) See a doctor."
        ],
        "choices_thai": [
            "(ไปเข้าแล็บเคมีทันที)",
            "(กินยาและพักผ่อน)",
            "(อ่านหนังสือเตรียมเข้าแล็บ)",
            "(ไปหาหมอ)"
        ],
        "answer_text": "เฉลย: (B)\nอธิบาย: คำแนะนำคือ 'take some aspirin' (กินยา) และ 'lie down' (นอนพัก/rest)"
    },
    {
        "id": 8,
        "narrator": "What implies about the assignment?",
        "script_eng": "(Woman): Have you started writing your essay yet?\n(Man): Started it? I handed it in two days ago!",
        "narrator_thai": "บทสนทนาบอกอะไรเกี่ยวกับงานเขียน?",
        "script_thai": "(Woman): คุณเริ่มเขียนเรียงความหรือยัง?\n(Man): เริ่มเหรอ? ผมส่งงานไปตั้งแต่ 2 วันที่แล้วแล้ว!",
        "choices_eng": [
            "(A) The man hasn't started it yet.",
            "(B) The man is writing it now.",
            "(C) The man finished it early.",
            "(D) The man lost the essay."
        ],
        "choices_thai": [
            "(ผู้ชายยังไม่ได้เริ่มทำ)",
            "(ผู้ชายกำลังเขียนอยู่)",
            "(ผู้ชายทำเสร็จก่อนกำหนด)",
            "(ผู้ชายทำเรียงความหาย)"
        ],
        "answer_text": "เฉลย: (C)\nอธิบาย: ผู้ชายบอกว่า 'handed it in two days ago' (ส่งไป 2 วันแล้ว) แสดงว่าเขาทำเสร็จเร็ว (Finished early) ก่อนที่ผู้หญิงจะถามเสียอีก"
    },
    {
        "id": 9,
        "narrator": "What does the woman mean?",
        "script_eng": "(Man): I can't believe we have to buy three textbooks for this course. They are so expensive.\n(Woman): Tell me about it. My wallet is empty.",
        "narrator_thai": "ผู้หญิงหมายความว่าอย่างไร?",
        "script_thai": "(Man): ไม่อยากเชื่อเลยว่าเราต้องซื้อหนังสือเรียนตั้ง 3 เล่มสำหรับวิชานี้ แพงชะมัด\n(Woman): เห็นด้วยสุดๆ กระเป๋าตังค์ฉันเกลี้ยงแล้วเนี่ย",
        "choices_eng": [
            "(A) She has lost her wallet.",
            "(B) She disagrees with the man.",
            "(C) She thinks the books are cheap.",
            "(D) She agrees that the books are expensive."
        ],
        "choices_thai": [
            "(เธอทำกระเป๋าตังค์หาย)",
            "(เธอไม่เห็นด้วยกับผู้ชาย)",
            "(เธอคิดว่าหนังสือราคาถูก)",
            "(เธอเห็นด้วยว่าหนังสือแพง)"
        ],
        "answer_text": "เฉลย: (D)\nอธิบาย: วลี 'Tell me about it' เป็นการแสดงความเห็นด้วยอย่างยิ่ง และการบอกว่า 'wallet is empty' ย้ำว่าหนังสือแพงจนเงินหมด"
    },
    {
        "id": 10,
        "narrator": "What will the man probably do?",
        "script_eng": "(Woman): The library is closing in five minutes. You'd better check those books out now.\n(Man): Oh, thanks for the warning. I'll head to the front desk right away.",
        "narrator_thai": "ผู้ชายน่าจะทำอะไร?",
        "script_thai": "(Woman): ห้องสมุดจะปิดในอีก 5 นาทีนะ คุณรีบไปยืมหนังสือพวกนั้นเดี๋ยวนี้เลยดีกว่า\n(Man): โอ้ ขอบคุณที่เตือน ผมจะรีบไปที่เคาน์เตอร์เดี๋ยวนี้เลย",
        "choices_eng": [
            "(A) Leave the books on the table.",
            "(B) Go to the front desk to borrow books.",
            "(C) Complain to the librarian.",
            "(D) Continue reading."
        ],
        "choices_thai": [
            "(ทิ้งหนังสือไว้บนโต๊ะ)",
            "(ไปที่เคาน์เตอร์เพื่อยืมหนังสือ)",
            "(ไปบ่นกับบรรณารักษ์)",
            "(อ่านหนังสือต่อ)"
        ],
        "answer_text": "เฉลย: (B)\nอธิบาย: ผู้ชายตอบรับว่าจะ 'head to the front desk' (มุ่งหน้าไปที่เคาน์เตอร์) เพื่อทำเรื่องยืมหนังสือ (check out)"
    }
]

In [5]:
# --- Helper Functions ---
def set_font_thai(run, font_name, font_size_pt, bold=False, color=None):
    run.font.name = font_name
    run.font.size = Pt(font_size_pt)
    run.font.bold = bold
    if color:
        run.font.color.rgb = color
    
    rPr = run._r.get_or_add_rPr()
    for type_attr in ['a:latin', 'a:ea', 'a:cs']:
        element = rPr.find(qn(type_attr))
        if element is None:
            element = OxmlElement(type_attr)
            rPr.append(element)
        element.set('typeface', font_name)

def set_cell_border(cell, border_color="000000", border_width='12700'):
    tc = cell._tc
    tcPr = tc.get_or_add_tcPr()
    for lines in ['a:lnL','a:lnR','a:lnT','a:lnB']:
        ln = OxmlElement(lines)
        ln.set('w', border_width)
        ln.set('cap', 'flat')
        ln.set('cmpd', 'sng')
        ln.set('algn', 'ctr')
        solidFill = OxmlElement('a:solidFill')
        srgbClr = OxmlElement('a:srgbClr')
        srgbClr.set('val', border_color)
        solidFill.append(srgbClr)
        ln.append(solidFill)
        prstDash = OxmlElement('a:prstDash')
        prstDash.set('val', 'solid')
        ln.append(prstDash)
        tcPr.append(ln)

def add_text_to_p(paragraph, text, size=18, bold=False, color=None):
    paragraph.text = "" 
    run = paragraph.add_run()
    run.text = text
    set_font_thai(run, FONT_NAME, size, bold, color)

# --- Add Question Slide ---
def add_question_slide(prs, q_data):
    slide_layout = prs.slide_layouts[6] 
    slide = prs.slides.add_slide(slide_layout)
    
    # Title
    txBox = slide.shapes.add_textbox(Inches(0.5), Inches(0.5), Inches(12), Inches(1.5))
    add_text_to_p(txBox.text_frame.paragraphs[0], f"Question {q_data['id']}", size=70, bold=True)
    
    # Narrator
    txBoxQ = slide.shapes.add_textbox(Inches(0.5), Inches(1.8), Inches(12), Inches(1.5))
    add_text_to_p(txBoxQ.text_frame.paragraphs[0], f"{q_data['narrator']}", size=50, bold=True)
    
    # Choices
    top = 3.2
    for choice in q_data['choices_eng']:
        txBoxC = slide.shapes.add_textbox(Inches(1), Inches(top), Inches(11), Inches(0.8))
        add_text_to_p(txBoxC.text_frame.paragraphs[0], choice, size=44)
        top += 0.9

# --- Add Answer Slide (Layout Fixed) ---
def add_answer_slide_formatted(prs, q_data):
    slide_layout = prs.slide_layouts[6]
    slide = prs.slides.add_slide(slide_layout)
    
    # Title
    txBox = slide.shapes.add_textbox(Inches(0.5), Inches(0.2), Inches(12), Inches(0.8))
    add_text_to_p(txBox.text_frame.paragraphs[0], f"Answer {q_data['id']}", size=60, bold=True)
    
    # Prepare Content
    eng_lines = q_data['script_eng'].split('\n')
    thai_lines = q_data['script_thai'].split('\n')
    eng_lines.append(f"(Narrator): {q_data['narrator']}")
    thai_lines.append(f"(Narrator): {q_data['narrator_thai']}")
    
    # --- Create Table with Fixed Position ---
    rows = 5
    cols = 4
    left = Inches(0.4)   # ขยับซ้ายเพื่อให้พื้นที่พอ
    top = Inches(1.2)    # ตำแหน่งความสูงเดิมตลอด
    width = Inches(12.5) # ความกว้างรวม
    height = Inches(6.0) # ความสูงรวม (Fix)
    
    shape = slide.shapes.add_table(rows, cols, left, top, width, height)
    table = shape.table
    
    # --- Set Fixed Column Widths ---
    # ตามสัดส่วน: Script/Thai = 4.0, Choices = 2.5, Answer = 2.0 (Answer is 1/2 of Script/Thai)
    table.columns[0].width = Inches(4.0) # Script
    table.columns[1].width = Inches(4.0) # Thai
    table.columns[2].width = Inches(2.5) # Choices
    table.columns[3].width = Inches(2.0) # Answer (เล็กกว่าเพื่อนครึ่งนึง)
    
    # --- Set Fixed Row Heights ---
    # Header 0.6", Content 1.3" each -> Total 5.8" (Fit in 7.5" slide)
    table.rows[0].height = Inches(0.6) 
    for r in range(1, 5):
        table.rows[r].height = Inches(1.35) 
        
    # Headers
    headers = ["Script", "คำแปลภาษาไทย", "ตัวเลือก", "เฉลยและคำอธิบาย"]
    for i, h_text in enumerate(headers):
        cell = table.cell(0, i)
        set_cell_border(cell)
        cell.fill.solid()
        cell.fill.fore_color.rgb = RGBColor(68, 114, 196)
        
        p = cell.text_frame.paragraphs[0]
        p.alignment = PP_ALIGN.CENTER
        cell.vertical_anchor = MSO_ANCHOR.MIDDLE
        add_text_to_p(p, h_text, size=24, bold=True, color=RGBColor(255, 255, 255))
        
    # Content
    choices_eng = q_data['choices_eng']
    choices_thai = q_data['choices_thai']
    
    for i in range(4):
        row_idx = i + 1 
        
        def set_content(cell, txt):
            set_cell_border(cell)
            cell.vertical_anchor = MSO_ANCHOR.TOP
            # ลด Font Content ลงเหลือ 18 เพื่อป้องกันการล้นช่องที่ Fix ไว้
            add_text_to_p(cell.text_frame.paragraphs[0], txt, size=18)

        # Col 0
        set_content(table.cell(row_idx, 0), eng_lines[i] if i < len(eng_lines) else "")
        
        # Col 1
        set_content(table.cell(row_idx, 1), thai_lines[i] if i < len(thai_lines) else "")

        # Col 2
        txt_choice = ""
        if i < len(choices_eng):
            txt_choice = f"{choices_eng[i]}\n{choices_thai[i]}"
        set_content(table.cell(row_idx, 2), txt_choice)
        
        # Col 3
        if i == 0:
            set_content(table.cell(row_idx, 3), q_data['answer_text'])
        else:
            set_content(table.cell(row_idx, 3), "")

    # Merge
    table.cell(1, 3).merge(table.cell(4, 3))

In [6]:
# --- Main ---
prs = Presentation()
prs.slide_width = Inches(13.333)
prs.slide_height = Inches(7.5)

# Title Slide
slide = prs.slides.add_slide(prs.slide_layouts[0])
add_text_to_p(slide.shapes.title.text_frame.paragraphs[0], "TOEFL ITP Practice", size=80, bold=True)
add_text_to_p(slide.placeholders[1].text_frame.paragraphs[0], "Part A: Short Conversations (Fixed Table Layout)", size=40)

for q in questions_data:
    add_question_slide(prs, q)
    add_answer_slide_formatted(prs, q)

prs.save("TOEFL_Listening_FixedLayout.pptx")
print("Done. Layout fixed.")

Done. Layout fixed.
